### Import packages that may need to be used in the future. 

In [4]:
import requests
from bs4 import BeautifulSoup
import time
import json
import pandas as pd
import requests_cache
import lxml.html as lx
import re

### Generate an access token that lasts for an hour. 

In [92]:
client_id = "21d31ff6fc1f41fea5b03cf985160b80"
client_secret = "b73e2df24c354dd69819e20cb7da5b86"

token_request_url = "https://accounts.spotify.com/api/token"
token_request_headers = {"Content-Type": "application/x-www-form-urlencoded"}
token_request_data = {
    "grant_type": "client_credentials",
    "client_id": client_id,
    "client_secret": client_secret
}

token_request_response = requests.post(url=token_request_url, headers=token_request_headers, data=token_request_data)

if token_request_response.status_code == 200:
    token_info = token_request_response.json()
    print(token_info)  # This will contain the access token
    access_token = token_info.get('access_token')
    print(f"Access token stored as: {access_token}")
else:
    print(f"Error: {token_request_response.status_code}, {token_request_response.text}")

{'access_token': 'BQAJdfCv9hAqiC6U-JBtWDEXFXZBBI8hWWT-x2V4O6uLqmOBDGohWMpJr_Bf2AkK1pp2sGcD5p1tiTaYLNrcP5DMFlP-TiGDPonk5kpKrN9Lq16nNI8-KnEO5OPXy-v4t1E8-kWnubQ', 'token_type': 'Bearer', 'expires_in': 3600}
Access token stored as: BQAJdfCv9hAqiC6U-JBtWDEXFXZBBI8hWWT-x2V4O6uLqmOBDGohWMpJr_Bf2AkK1pp2sGcD5p1tiTaYLNrcP5DMFlP-TiGDPonk5kpKrN9Lq16nNI8-KnEO5OPXy-v4t1E8-kWnubQ


### Function to get the artist's info given a link to the artist. 

In [27]:
def get_artist_info(artist_url):
    url = artist_url
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        artist_data = response.json()
        print(artist_data)  # Prints the artist's details
    else:
        print(f"Error: {response.status_code}, {response.text}")

get_artist_info("https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ") # The Weeknd https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ?si=HhB2mjsUSQ-5oYN7A9i6ZA
get_artist_info("https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s") # Justin Bieber https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s?si=qXJYi_CdTri2kj_3d1_HHg


{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpatF05PJ'}, 'followers': {'href': None, 'total': 101372361}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/1Xyo4u8uXC1ZmMpatF05PJ', 'id': '1Xyo4u8uXC1ZmMpatF05PJ', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb9e528993a2820267b97f6aae', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab676161000051749e528993a2820267b97f6aae', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f1789e528993a2820267b97f6aae', 'height': 160, 'width': 160}], 'name': 'The Weeknd', 'popularity': 97, 'type': 'artist', 'uri': 'spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ'}
{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'}, 'followers': {'href': None, 'total': 81222845}, 'genres': [], 'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s', 'id': '1uNFoZAHBGtllmzznpCI3s', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5e

### Function to obtain all data on a playlist. 

In [41]:
def get_playlist_info(playlist_url):
    url = playlist_url
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        playlist_data = response.json()
        print(playlist_data)  # Prints the playlist's details
    else:
        print(f"Error: {response.status_code}, {response.text}")

    """
    Save song 'name','release_date', 'release_date_precision', 'id', 'external_ids', artist 'name',  'id', 'popularity'
    """

get_playlist_info("https://api.spotify.com/v1/playlists/6UeSakyzhiEt4NB3UAd6NQ") # https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ?si=b65f446ee0174c78

{'collaborative': False, 'description': 'The official <a href="http://www.billboard.com/charts/hot-100">Billboard Hot 100</a> features this week\'s most popular songs across all genres, ranked by radio airplay monitored by Nielsen BDS, download sales tracked by Nielsen SoundScan and streaming activity data provided by leading online music services. For more information go to <a href="http://Billboard.com">Billboard.com</a>.', 'external_urls': {'spotify': 'https://open.spotify.com/playlist/6UeSakyzhiEt4NB3UAd6NQ'}, 'followers': {'href': None, 'total': 1804606}, 'href': 'https://api.spotify.com/v1/playlists/6UeSakyzhiEt4NB3UAd6NQ', 'id': '6UeSakyzhiEt4NB3UAd6NQ', 'images': [{'height': None, 'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000d72c3bd5501a335b265807df34db', 'width': None}], 'name': 'Billboard Hot 100', 'owner': {'display_name': 'Billboard', 'external_urls': {'spotify': 'https://open.spotify.com/user/billboard.com'}, 'href': 'https://api.spotify.com/v1/users/billb

### Function that gets the useful information of each track on a playlist. 

In [33]:
def get_playlist_tracks_info(playlist_url, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.get(playlist_url, headers=headers)
    
    if response.status_code == 200:
        playlist_data = response.json()
        
        extracted_tracks = []
        
        for item in playlist_data.get('tracks', {}).get('items', []):
            track = item.get('track', {})
            
            if track:
                track_info = {
                    "song_name": track.get('name'),
                    "song_id": track.get('id'),
                    "release_date": track.get('album', {}).get('release_date'),
                    "release_date_precision": track.get('album', {}).get('release_date_precision'),
                    "external_ids": track.get('external_ids', {}),
                    "artists": [
                        {
                            "artist_name": artist.get('name'),
                            "artist_id": artist.get('id'),
                            "popularity": track.get('popularity')
                        }
                        for artist in track.get('artists', [])
                    ]
                }
                extracted_tracks.append(track_info)
        
        return extracted_tracks

    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None

playlist_url = "https://api.spotify.com/v1/playlists/6UeSakyzhiEt4NB3UAd6NQ"
playlist_info = get_playlist_tracks_info(playlist_url, access_token)

if playlist_info:
    for track in playlist_info:
        print(track)

{'song_name': 'luther (with sza)', 'song_id': '45J4avUb9Ni0bnETYaYFVJ', 'release_date': '2024-11-22', 'release_date_precision': 'day', 'external_ids': {'isrc': 'USUG12408496'}, 'artists': [{'artist_name': 'Kendrick Lamar', 'artist_id': '2YZyLoL8N0Wb9xBt1NhZWg', 'popularity': 92}, {'artist_name': 'SZA', 'artist_id': '7tYKF4w9nC0nq9CsPZTHyP', 'popularity': 92}]}
{'song_name': 'Die With A Smile', 'song_id': '2plbrEY59IikOBgBGLjaoe', 'release_date': '2024-08-16', 'release_date_precision': 'day', 'external_ids': {'isrc': 'USUM72409273'}, 'artists': [{'artist_name': 'Lady Gaga', 'artist_id': '1HY2Jd0NmPuamShAr6KMms', 'popularity': 100}, {'artist_name': 'Bruno Mars', 'artist_id': '0du5cEVh5yTK9QJze8zA0C', 'popularity': 100}]}
{'song_name': 'Not Like Us', 'song_id': '6AI3ezQ4o3HUoP6Dhudph3', 'release_date': '2024-05-04', 'release_date_precision': 'day', 'external_ids': {'isrc': 'USUG12400910'}, 'artists': [{'artist_name': 'Kendrick Lamar', 'artist_id': '2YZyLoL8N0Wb9xBt1NhZWg', 'popularity': 9

### Function for Search Query with Spotify API 

In [35]:
"""
Input: 
playlist = [(song_name, artist_name), ...]
"""

def clean_artist_name(artist_name):
    # Removes "with" and anything after it in artist name to handle features (most popular version should be the version we are looking for) 
    return re.sub(r"\swith.*", "", artist_name).strip()

def clean_song_name(song_name):
    # Removes text inside and including parentheses to handle versions (most popular version should be the version we are looking for) 
    return re.sub(r"\s*\(.*?\)", "", song_name).strip()

def search_spotify(playlist, access_token):
    url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {access_token}"}
    results = []

    def find_most_popular_track(tracks):
        # Sorts tracks by popularity and returns the most popular track's Spotify ID 
        if not tracks:
            return None
        tracks.sort(key=lambda x: x.get("popularity", 0), reverse=True)
        return tracks[0]["id"]

    for song_name, artist_name in playlist:
        # Clean song and artist names
        song_name = clean_song_name(song_name)
        artist_name = clean_artist_name(artist_name)

        params = {
            "q": f'track:\"{song_name}\" artist:\"{artist_name}\"',
            "type": "track",
            "limit": 10
        }

        try:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()

            tracks = data.get("tracks", {}).get("items", [])
            spotify_track_id = find_most_popular_track(tracks)
            
            if spotify_track_id:
                results.append({
                    "song_name": song_name,
                    "artist_name": artist_name,
                    "spotify_track_id": spotify_track_id
                })
            else:
                print(f"No results found for: {song_name} by {artist_name}")

        except requests.exceptions.RequestException as e:
            print(f"Error fetching {song_name} by {artist_name}: {e}")

        time.sleep(0.1)

    return results

# Testing 
results = search_spotify(formatted_songs_70s, access_token)
print(results)

No results found for: Another Brick in the Wall Part II by Pink Floyd
No results found for: Free Bird by Lynyrd Skynrd
[{'song_name': 'Bohemian Rhapsody', 'artist_name': 'Queen', 'spotify_track_id': '7tFiyTwD0nx5a1eklYtX2J'}, {'song_name': 'Imagine', 'artist_name': 'John Lennon', 'spotify_track_id': '7pKfPomDEeI4TPT6EOYjn9'}, {'song_name': 'Stayin’ Alive', 'artist_name': 'Bee Gees', 'spotify_track_id': '3mRM4NM8iO7UBqrSigCQFH'}, {'song_name': 'Bridge Over Troubled Water', 'artist_name': 'Simon & Garfunkel', 'spotify_track_id': '6l8EbYRtQMgKOyc1gcDHF9'}, {'song_name': 'Hotel California', 'artist_name': 'Eagles', 'spotify_track_id': '40riOy7x9W7GXjyGp4pjAv'}, {'song_name': 'American Pie', 'artist_name': 'Don McLean', 'spotify_track_id': '1fDsrQ23eTAVFElUMaf38X'}, {'song_name': 'Stairway to Heaven', 'artist_name': 'Led Zeppelin', 'spotify_track_id': '5CQ30WqJwcep0pYcV4AMNc'}, {'song_name': 'Let It Be', 'artist_name': 'The Beatles', 'spotify_track_id': '7iN1s7xHE4ifF5povM6A48'}, {'song_nam

#### Create lists and find songs that did not yield a result during the search query 

In [72]:
daves_top_100_1970s, daves_top_100_1980s, daves_top_100_1990s, daves_top_100_2000s, daves_top_100_2010s = [], [], [], [], []

print("Generating 70s...")
daves_top_100_1970s = search_spotify(formatted_songs_70s, access_token)
print("Generating 80s...")
daves_top_100_1980s = search_spotify(formatted_songs_80s, access_token)
print("Generating 90s...")
daves_top_100_1990s = search_spotify(formatted_songs_90s, access_token)
print("Generating 2000s...")
daves_top_100_2000s = search_spotify(formatted_songs_2000s, access_token)
print("Generating 2010s...")
daves_top_100_2010s = search_spotify(formatted_songs_2010s, access_token)



Generating 70s...
No results found for: Another Brick in the Wall Part II by Pink Floyd
No results found for: Free Bird by Lynyrd Skynrd
Generating 80s...
No results found for: I Love Rock and Roll by Joan Jett & the Blackhearts
No results found for: That's What Friends Are For by Dionne & Friends (Dionne Warwick
Generating 90s...
Generating 2000s...
No results found for: Bring Me to Life by Evanesence
No results found for: Rik Rok by Shaggy wih Ricardo
Generating 2010s...
No results found for: Fuck You (aka by Cee-Lo Green


#### Manually add the songs if they exist on Spotify 

In [74]:
# Function to get the basic track info for missing tracks 
def get_basic_track_info(track_url, access_token):
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.get(track_url, headers=headers)
    
    if response.status_code == 200:
        try:
            data = response.json()
            
            # Extract relevant details
            track_info = {
                "spotify_track_name": data.get("name"),
                "spotify_artist_name": data["artists"][0]["name"] if data.get("artists") else "Unknown",
                "spotify_track_id": data.get("id")
            }
            
            return track_info

        except requests.exceptions.RequestException as e:
            print(f"Error processing response: {e}")
    
    else:
        print(f"Error: {response.status_code}, {response.text}")
    time.sleep(0.5)
    return None

# Testing (https://open.spotify.com/track/4gMgiXfqyzZLMhsksGmbQV?si=0c02beed56954203) 
track_url = "https://api.spotify.com/v1/tracks/4gMgiXfqyzZLMhsksGmbQV"
track_info = get_basic_track_info(track_url, access_token)
print(track_info)

{'spotify_track_name': 'Another Brick in the Wall, Pt. 2', 'spotify_artist_name': 'Pink Floyd', 'spotify_track_id': '4gMgiXfqyzZLMhsksGmbQV'}


In [78]:
# Add the tracks to their respective playlists 

## 70s 
# Another Brick in the Wall Part II by Pink Floyd (https://open.spotify.com/track/4gMgiXfqyzZLMhsksGmbQV?si=0c02beed56954203) 
track_url = "https://api.spotify.com/v1/tracks/4gMgiXfqyzZLMhsksGmbQV"
track_info = get_basic_track_info(track_url, access_token)
daves_top_100_1970s.append(track_info)
# Free Bird by Lynyrd Skynrd (https://open.spotify.com/track/5EWPGh7jbTNO2wakv8LjUI?si=a84bfd14b8b0402e) 
track_url = "https://api.spotify.com/v1/tracks/5EWPGh7jbTNO2wakv8LjUI"
track_info = get_basic_track_info(track_url, access_token)
daves_top_100_1970s.append(track_info)

## 80s
# I Love Rock and Roll by Joan Jett & the Blackhearts (https://open.spotify.com/track/2Cdvbe2G4hZsnhNMKyGrie?si=7e6a3f5b495146d7)
track_url = "https://api.spotify.com/v1/tracks/2Cdvbe2G4hZsnhNMKyGrie"
track_info = get_basic_track_info(track_url, access_token)
daves_top_100_1980s.append(track_info)

# That's What Friends Are For by Dionne & Friends (https://open.spotify.com/track/1cOj24fUMnBSXhtp7waHVW?si=55506bffd88745df)
track_url = "https://api.spotify.com/v1/tracks/1cOj24fUMnBSXhtp7waHVW"
track_info = get_basic_track_info(track_url, access_token)
daves_top_100_1980s.append(track_info)

## 2000s
# Bring Me to Life by Evanesence (https://open.spotify.com/track/0COqiPhxzoWICwFCS4eZcp?si=e1360638e623457b)
track_url = "https://api.spotify.com/v1/tracks/0COqiPhxzoWICwFCS4eZcp"
track_info = get_basic_track_info(track_url, access_token)
daves_top_100_2000s.append(track_info)
# Rik Rok by Shaggy wih Ricardo (https://open.spotify.com/track/3WkibOpDF7cQ5xntM1epyf?si=6375abf8427d4f6c) 
track_url = "https://api.spotify.com/v1/tracks/3WkibOpDF7cQ5xntM1epyf"
track_info = get_basic_track_info(track_url, access_token)
daves_top_100_2000s.append(track_info)

## 2010s
# F*** You by Cee-Lo Green (https://open.spotify.com/track/4ycLiPVzE5KamivXrAzGFG?si=362e1a729be2402d) 
track_url = "https://api.spotify.com/v1/tracks/4ycLiPVzE5KamivXrAzGFG"
track_info = get_basic_track_info(track_url, access_token)
daves_top_100_2010s.append(track_info)


In [90]:
top_100_lists = [daves_top_100_1970s, daves_top_100_1980s, daves_top_100_1990s, daves_top_100_2000s, daves_top_100_2010s]
year = 1970
for list in top_100_lists:
    print(f"Era: {year}s \n{list}")
    print(f"Tracks in list: {len(list)} \n\n\n")
    year += 10

Era: 1970s 
[{'song_name': 'Bohemian Rhapsody', 'artist_name': 'Queen', 'spotify_track_id': '7tFiyTwD0nx5a1eklYtX2J'}, {'song_name': 'Imagine', 'artist_name': 'John Lennon', 'spotify_track_id': '7pKfPomDEeI4TPT6EOYjn9'}, {'song_name': 'Stayin’ Alive', 'artist_name': 'Bee Gees', 'spotify_track_id': '3mRM4NM8iO7UBqrSigCQFH'}, {'song_name': 'Bridge Over Troubled Water', 'artist_name': 'Simon & Garfunkel', 'spotify_track_id': '6l8EbYRtQMgKOyc1gcDHF9'}, {'song_name': 'Hotel California', 'artist_name': 'Eagles', 'spotify_track_id': '40riOy7x9W7GXjyGp4pjAv'}, {'song_name': 'American Pie', 'artist_name': 'Don McLean', 'spotify_track_id': '1fDsrQ23eTAVFElUMaf38X'}, {'song_name': 'Stairway to Heaven', 'artist_name': 'Led Zeppelin', 'spotify_track_id': '5CQ30WqJwcep0pYcV4AMNc'}, {'song_name': 'Let It Be', 'artist_name': 'The Beatles', 'spotify_track_id': '7iN1s7xHE4ifF5povM6A48'}, {'song_name': 'Layla', 'artist_name': 'Derek & the Dominos', 'spotify_track_id': '2kkvB3RNRzwjFdGhaUA0tz'}, {'song_na

In [104]:
def fetch_track_details(track_list, access_token):
    # Fetch detailed track info from Spotify API and store results in a pandas DataFrame 
    extracted_tracks = []
    headers = {
        "Authorization": f"Bearer {access_token}"
    }

    for track in track_list:
        track_id = track.get("spotify_track_id")
        if not track_id:
            continue 

        ## Track details 
        track_url = f"https://api.spotify.com/v1/tracks/{track_id}"
        response = requests.get(track_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()

            # Extract relevant details
            track_info = {
                "song_name": data.get("name"),
                "song_id": data.get("id"),
                "popularity": data.get("popularity"),
                "release_date": data.get("album", {}).get("release_date"),
                "release_date_precision": data.get("album", {}).get("release_date_precision"),
                "external_ids": data.get("external_ids", {}),
                "spotify_url": data["external_urls"]["spotify"],
                "artists": [
                    {
                        "artist_name": artist.get("name"),
                        "artist_id": artist.get("id")
                    }
                    for artist in data.get("artists", [])
                ]
            }
            extracted_tracks.append(track_info)
        else:
            print(f"(track details) Error fetching track {track_id}: {response.status_code}, {response.text}")

        """
        ## Track audio features 
        track_url = f"https://api.spotify.com/v1/audio-features/{track_id}"
        response = requests.get(track_url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()

            # Extract relevant details
            track_info = {
                "acousticness": data.get("namacousticnesse"),
                "danceability": data.get("danceability"),
                "energy": data.get("energy"),
                "instrumentalness": data.get("instrumentalness"),
                "liveness": data.get("liveness"),
                "loudness": data.get("loudness"),
                "speechiness": data.get("speechiness"),
                "tempo": data.get("tempo"),
                "valence": data.get("valence"),
                "duration_ms": data.get("duration_ms")
            }
            extracted_tracks.append(track_info)
        else:
            print(f"(audio analysis) Error fetching track {track_id}: {response.status_code}, {response.text}")

        time.sleep(0.05)
        """


    # Convert extracted data into a Pandas DataFrame
    df = pd.DataFrame(extracted_tracks)
    return df

daves_top_100_1970s_df, daves_top_100_1980s_df, daves_top_100_1990s_df, daves_top_100_2000s_df, daves_top_100_2010s_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame() 

top_100_dataFrames = [daves_top_100_1970s_df, daves_top_100_1980s_df, daves_top_100_1990s_df, daves_top_100_2000s_df, daves_top_100_2010s_df]

year = 1970
for top_100_dataFrame, top_100_list in zip(top_100_dataFrames, top_100_lists):
    top_100_dataFrame = fetch_track_details(top_100_list, access_token)
    print(f"Era: {year}s \n{top_100_dataFrame}")
    print(f"Tracks in list: {len(top_100_dataFrame)} \n\n\n")
    year += 10

Era: 1970s 
                                            song_name                 song_id  \
0                 Bohemian Rhapsody - Remastered 2011  7tFiyTwD0nx5a1eklYtX2J   
1                           Imagine - Remastered 2010  7pKfPomDEeI4TPT6EOYjn9   
2   Stayin' Alive - From "Saturday Night Fever" So...  3mRM4NM8iO7UBqrSigCQFH   
3                          Bridge Over Troubled Water  6l8EbYRtQMgKOyc1gcDHF9   
4                    Hotel California - 2013 Remaster  40riOy7x9W7GXjyGp4pjAv   
..                                                ...                     ...   
95           Old Time Rock And Roll - Remastered 2011  5jQI0mj9xWX6m8TymhzWll   
96                              Da Ya Think I'm Sexy?  4z7maGZkAonDlXlwo8q69f   
97              Bennie And The Jets - Remastered 2014  5Wj1rJnCLpMHdLaxsFtJLs   
98                   Another Brick in the Wall, Pt. 2  4gMgiXfqyzZLMhsksGmbQV   
99                                          Free Bird  5EWPGh7jbTNO2wakv8LjUI   

    popularity 

## Retreive Top 100 1970s Songs from Dave's Music Database

In [11]:
# full 1970s songs

# Set up a cached session
session = requests_cache.CachedSession()

# 1970s url
dave_1970_url = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1970-1979.html'
time.sleep(0.05)

# Get request
r1 = session.get(dave_1970_url)
r1.raise_for_status()

# Navigate all the songs in the a tags
html1 = lx.fromstring(r1.text)
artist_songs_1970s = html1.xpath('//p//a/text() | //p/text()')

# Drop the extra text in the beginning and end
artist_songs_1970s_ = artist_songs_1970s[4:-6]

# Clean up the list, get rid of whitespace charcters
# Remove extra text in the beginning and end
cleaned_text_1970s = ' '.join(artist_songs_1970s_)
cleaned_text_1970s = ' '.join(cleaned_text_1970s.split())
rankings_start_1970s = cleaned_text_1970s.find("1.")  # Find the start of the rankings
rankings_text_1970s = cleaned_text_1970s[rankings_start_1970s:]

# Split on ranking
# Remove extra space in quotations
songs_1970s = re.split(r'\d+\.', rankings_text_1970s)
cleaned_songs_1970s = [song.strip().replace("“ ", "“").replace(" ”", "”") for song in songs_1970s if song.strip()] 

# Create a list of tuples with (song title, artist)
# Extract the song title between the double quotations
# Extract the artist which comes before "
formatted_songs_70s = [(song.split('“')[1].split('”')[0], song.split('“')[0].strip()) for song in cleaned_songs_1970s]
formatted_songs_70s

[('Bohemian Rhapsody', 'Queen'),
 ('Imagine', 'John Lennon'),
 ('Stayin’ Alive', 'Bee Gees'),
 ('Bridge Over Troubled Water', 'Simon & Garfunkel'),
 ('Hotel California', 'Eagles'),
 ('American Pie', 'Don McLean'),
 ('Stairway to Heaven', 'Led Zeppelin'),
 ('Let It Be', 'The Beatles'),
 ('Layla', 'Derek & the Dominos'),
 ('Superstition', 'Stevie Wonder'),
 ('I Will Survive', 'Gloria Gaynor'),
 ('Dancing Queen', 'Abba'),
 ('Another Brick in the Wall Part II', 'Pink Floyd'),
 ('Born to Run', 'Bruce Springsteen'),
 ('What’s Going On', 'Marvin Gaye'),
 ('Maggie May', 'Rod Stewart'),
 ('Your Song', 'Elton John'),
 ('Let’s Stay Together', 'Al Green'),
 ('Heart of Glass', 'Blondie'),
 ('Le Freak', 'Chic'),
 ('My Sweet Lord', 'George Harrison'),
 ('London Calling', 'The Clash'),
 ('Sweet Home Alabama', 'Lynyrd Skynyrd'),
 ('Y.M.C.A.', 'Village People'),
 ('Let’s Get It On', 'Marvin Gaye'),
 ('How Deep Is Your Love', 'Bee Gees'),
 ('You’re the One That I Want', 'Olivia Newton-John & John Travolt

## Retreive Top 100 1980s Songs from Dave's Music Database

In [17]:
# full 80's songs 

# Set up a cached session
session = requests_cache.CachedSession()

# 80s url
dave_1980_url = 'https://davesmusicdatabase.blogspot.com/2014/08/top-100-songs-from-1980-to-1989.html'
time.sleep(0.05)

# Get request
r2 = session.get(dave_1980_url)
r2.raise_for_status()

# Navigate all the songs in the a tags
# Remove beginning and ending message
html2 = lx.fromstring(r2.text)
artist_songs_80s = html2.xpath('//p//a/text() | //p/text()')
artist_songs_80s_ = artist_songs_80s[10:-4]

# Clean up the list, get rid of whitespace charcters
# Remove extra text in the beginning and end
cleaned_text_80s = ' '.join(artist_songs_80s_)
cleaned_text_80s = ' '.join(cleaned_text_80s.split())

# Find the start of the rankings
rankings_start_80s = cleaned_text_80s.find("1.") 
rankings_text_80s = cleaned_text_80s[rankings_start_80s:]

# split songs based on ranking
# use list generator to remove extra spaces
songs_80s = re.split(r'\d+\.', rankings_text_80s)
cleaned_songs_80s = [song.strip().replace("“ ", "“").replace(" ”", "”") for song in songs_80s if song.strip()]  
#cleaned_songs_80s

# Create a list of tuples with (song title, artist)
# Extract the song title between the double quotations
# Extract the artist which comes before "
formatted_songs_80s = [(song.split('“')[1].split('”')[0], song.split('“')[0].strip()) for song in cleaned_songs_80s]
formatted_songs_80s

[('Billie Jean', 'Michael Jackson'),
 ('Every Breath You Take', 'The Police'),
 ('Sweet Child O’ Mine', 'Guns N’ Roses'),
 ('When Doves Cry', 'Prince'),
 ('I Love Rock and Roll', 'Joan Jett & the Blackhearts'),
 ('With or Without You', 'U2'),
 ('Livin’ on a Prayer', 'Bon Jovi'),
 ('Endless Love', 'Lionel Richie & Diana Ross'),
 ('Jump', 'Van Halen'),
 ('Beat It', 'Michael Jackson'),
 ('You Shook Me All Night Long', 'AC/DC'),
 ('Love Will Tear Us Apart', 'Joy Division'),
 ('Don’t Stop Believin’', 'Journey'),
 ('Eye of the Tiger', 'Survivor'),
 ('We Are the World', 'U.S.A. for Africa'),
 ('Don’t You Want Me?', 'The Human League'),
 ('Like a Prayer', 'Madonna'),
 ('Tainted Love', 'Soft Cell'),
 ('I Wanna Dance with Somebody Who Loves Me', 'Whitney Houston'),
 ('Bette Davis Eyes', 'Kim Carnes'),
 ('Another One Bites the Dust', 'Queen'),
 ('Celebration', 'Kool & the Gang'),
 ('Take on Me', 'Aha'),
 ('Call Me', 'Blondie'),
 ('Blue Monday', 'New Order'),
 ('I Still Haven’t Found What I’m Look

## Retreive Top 100 1990s Songs from Dave's Music Database

In [19]:
# full 90s songs

# 90s url
dave_1990_url = 'https://davesmusicdatabase.blogspot.com/2014/08/top-songs-from-1990-to-1999.html'
time.sleep(0.05)

# Get request
r3 = session.get(dave_1990_url)
r3.raise_for_status()

# Navigate all the songs in the a tags
html3 = lx.fromstring(r3.text)
artist_songs_90s = html3.xpath('//p//a/text() | //p/text()')

# Clean up the list, get rid of whitespace charcters
# Remove extra text in the beginning and end
cleaned_text_90s = ' '.join(artist_songs_90s)
cleaned_text_90s = ' '.join(cleaned_text_90s.split())
rankings_start_90s = cleaned_text_90s.find("1.")  # Find the start of the rankings
rankings_text_90s = cleaned_text_90s[rankings_start_90s:]

# Split on ranking
# Remove extra space in quotations
songs_90s = re.split(r'\d+\.', rankings_text_90s)
cleaned_songs_90s = [song.strip().replace("“ ", "“").replace(" ”", "”") for song in songs_90s if song.strip()]  
#cleaned_songs_90s

# Create a list of tuples with (song title, artist)
# Extract the song title between the double quotations
# Extract the artist which comes before "
formatted_songs_90s = [(song.split('“')[1].split('”')[0], song.split('“')[0].strip()) for song in cleaned_songs_90s]
formatted_songs_90s

[('I Will Always Love You', 'Whitney Houston'),
 ('Smells Like Teen Spirit', 'Nirvana'),
 ('(Everything I Do) I Do It for You', 'Bryan Adams'),
 ('Nothing Compares 2 U', 'Sinéad O’Connor'),
 ('My Heart Will Go On', 'Celine Dion'),
 ('Losing My Religion', 'R.E.M.'),
 ('Candle in the Wind 1997 (Goodbye England’s Rose)', 'Elton John'),
 ('Wonderwall', 'Oasis'),
 ('Macarena (Bayside Boys Mix)', 'Los Del Rio'),
 ('One', 'U2'),
 ('Gangsta’s Paradise', 'Coolio with L.V.'),
 ('Creep', 'Radiohead'),
 ('Baby One More Time', 'Britney Spears'),
 ('Bitter Sweet Symphony', 'The Verve'),
 ('Wannabe', 'Spice Girls'),
 ('Under the Bridge', 'Red Hot Chili Peppers'),
 ('Smooth', 'Santana with Rob Thomas'),
 ('I Don’t Want to Miss a Thing', 'Aerosmith'),
 ('I’ll Make Love to You', 'Boyz II Men'),
 ('All I Want for Christmas Is You', 'Mariah Carey'),
 ('Iris', 'Goo Goo Dolls'),
 ('I’ll Be Missing You', 'Puff Daddy with Faith Evans & 112'),
 ('Believe', 'Cher'),
 ('You’re Still the One', 'Shania Twain'),
 (

## Retreive Top 100 2000s Songs Dave's Music Database

In [23]:
# full 2000s songs

# 2000s url
dave_2000_url = 'https://davesmusicdatabase.blogspot.com/2017/10/the-top-100-songs-from-2000-2009.html'
time.sleep(0.05)

# Get request
r4 = session.get(dave_2000_url)
r4.raise_for_status()

# Navigate all the songs in the a tags
html4 = lx.fromstring(r4.text)
artist_songs_2000s = html4.xpath('//p//a/text() | //p/text()')

# Drop the extra text in the beginning and end
artist_songs_2000s_ = artist_songs_2000s[4:-18]
artist_songs_2000s_

# Clean up the list, get rid of whitespace charcters
# Remove extra text in the beginning and end
cleaned_text_2000s = ' '.join(artist_songs_2000s_)
cleaned_text_2000s = ' '.join(cleaned_text_2000s.split())
rankings_start_2000s = cleaned_text_2000s.find("1.")  # Find the start of the rankings
rankings_text_2000s = cleaned_text_2000s[rankings_start_2000s:]

# Split on ranking
# Remove extra space in quotations
songs_2000s = re.split(r'\d+\.', rankings_text_2000s)
cleaned_songs_2000s = [song.strip().replace("“ ", "“").replace(" ”", "”") for song in songs_2000s if song.strip()]  

# Create a list of tuples with (song title, artist)
# Extract the song title between the double quotations
# Extract the artist which comes before "
formatted_songs_2000s = [(song.split('“')[1].split('”')[0], song.split('“')[0].strip()) for song in cleaned_songs_2000s]
formatted_songs_2000s

[('Hey Ya!', 'OutKast'),
 ('I Gotta Feeling', 'Black Eyed Peas'),
 ('Lose Yourself', 'Eminem'),
 ('Yeah!', 'Usher with Lil’ Jon & Ludacris'),
 ('Crazy in Love', 'Beyoncé with Jay-Z'),
 ('Crazy', 'Gnarls Barkley'),
 ('Single Ladies (Put a Ring on It)', 'Beyoncé'),
 ('Umbrella', 'Rihanna with Jay-Z'),
 ('Low', 'Flo Rida with T-Pain'),
 ('We Belong Together', 'Mariah Carey'),
 ('Viva La Vida', 'Coldplay'),
 ('Poker Face', 'Lady Gaga'),
 ('Boom Boom Pow', 'Black Eyed Peas'),
 ('Tik Tok', 'Ke$ha'),
 ('Gold Digger', 'Kanye West with Jamie Foxx'),
 ('Bleeding Love', 'Leona Lewis'),
 ('Just Dance', 'Lady Gaga with Colby O’Donnis'),
 ('I’m Yours', 'Jason Mraz'),
 ('In Da Club', '50 Cent'),
 ('Irreplaceable', 'Beyoncé'),
 ('Sexyback', 'Justin Timberlake'),
 ('Hips Don’t Lie', 'Shakira with Wyclef Jean'),
 ('Hot in Herre', 'Nelly'),
 ('Fallin’', 'Alicia Keys'),
 ('You’re Beautiful', 'James Blunt'),
 ('No One', 'Alicia Keys'),
 ('Can’t Get You Out of My Head', 'Kylie Minogue'),
 ('I Kissed a Girl'

## Retreive Top 100 2010s Songs from Dave's Music Database

In [25]:
# full 2010s songs

# 2010s url
dave_2010_url = 'https://davesmusicdatabase.blogspot.com/2017/10/the-top-50-songs-from-2010-2016.html'
time.sleep(0.05)

# Get request
r5 = session.get(dave_2010_url)
r5.raise_for_status()

# Navigate all the songs in the a tags
html5 = lx.fromstring(r5.text)
artist_songs_2010s = html5.xpath('//p//a/text() | //p/text()')

# Drop the extra text in the beginning and end
artist_songs_2010s_ = artist_songs_2010s[5:-5]

# Clean up the list, get rid of whitespace charcters
# Remove extra text in the beginning and end
cleaned_text_2010s = ' '.join(artist_songs_2010s_)
cleaned_text_2010s = ' '.join(cleaned_text_2010s.split())
rankings_start_2010s = cleaned_text_2010s.find("1.")  # Find the start of the rankings
rankings_text_2010s = cleaned_text_2010s[rankings_start_2010s:]

# Split on ranking
# Remove extra space in quotations
songs_2010s = re.split(r'\d+\.', rankings_text_2010s)
cleaned_songs_2010s = [song.strip().replace("“ ", "“").replace(" ”", "”") for song in songs_2010s if song.strip()]  

# Create a list of tuples with (song title, artist)
# Extract the song title between the double quotations
# Extract the artist which comes before "
formatted_songs_2010s = [(song.split('“')[1].split('”')[0], song.split('“')[0].strip()) for song in cleaned_songs_2010s]
formatted_songs_2010s

[('Uptown Funk!', 'Mark Ronson with Bruno Mars'),
 ('Rolling in the Deep', 'Adele'),
 ('Shape of You', 'Ed Sheeran'),
 ('Happy', 'Pharrell Williams'),
 ('Somebody That I Used to Know', 'Gotye with Kimbra'),
 ('Despacito', 'Luis Fonsi with Daddy Yankee & Justin Bieber'),
 ('Call Me Maybe', 'Carly Rae Jepsen'),
 ('Blurred Lines', 'Robin Thicke with T.I. & Pharrell Williams'),
 ('Old Town Road', 'Lil Nas X with Billy Ray Cyrus'),
 ('Hello', 'Adele'),
 ('We Found Love', 'Rihanna with Calvin Harris'),
 ('Party Rock Anthem', 'LMFAO with Lauren Bennett & GoonRock'),
 ('Closer', 'The Chainsmokers with Halsey'),
 ('Blinding Lights', 'The Weeknd'),
 ('Royals', 'Lorde'),
 ('Thinking Out Loud', 'Ed Sheeran'),
 ('See You Again', 'Wiz Khalifa with Charlie Puth'),
 ('Perfect', 'Ed Sheeran with Beyoncé'),
 ('Just the Way You Are (Amazing)', 'Bruno Mars'),
 ('Moves Like Jagger', 'Maroon 5 with Christina Aguilera'),
 ('Someone Like You', 'Adele'),
 ('Shake It Off', 'Taylor Swift'),
 ('Love the Way You L